# Notebook 01: Entrenamiento y Validación del Modelo Core (GBR)
## Objetivo:

- Entrenar un modelo base (Gradient Boosting) para predecir la media de la demanda.

- Validar el rendimiento de este modelo contra los datos de 2023 y compararlo con el modelo_actual.

- Guardar el modelo entrenado y los datos de validación para los siguientes notebooks.

## 0. Configuración e Importación de Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib # Para guardar el modelo

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Configuración de visualización
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 7)

## 1. Carga y Exploración de Datos (EDA)


In [ ]:
# Cargar el dataset (ajusta la ruta según tu estructura)
path_datos = '../data/raw/demanding_forecast.csv'

df = pd.read_csv(path_datos)



In [ ]:
# Convertir la columna 'fecha' a datetime
df['fecha'] = pd.to_datetime(df['fecha'])

print(df.info())
df.head()

In [ ]:
# Resumen estadístico
num_productos = df['prod_id'].nunique()
fecha_min = df['fecha'].min()
fecha_max = df['fecha'].max()

print(f"Número de productos únicos: {num_productos}")
print(f"Rango de fechas: {fecha_min.date()} a {fecha_max.date()}")

## Análisis de la Variable Objetivo (ventas)
Buscamos evidencia de "colas pesadas" (eventos extremos) que justifiquen nuestro modelo de riesgo (GPD).

In [ ]:
plt.figure(figsize=(15, 7))
sns.boxplot(x=df['ventas'], fliersize=2)
plt.title('Distribución de Ventas (Outliers y Colas Pesadas)')
plt.xlabel('Ventas')
plt.show()

plt.figure(figsize=(15, 7))
sns.histplot(df['ventas'], bins=100, kde=True)
plt.title('Histograma de Ventas')
plt.show()

Observación de EDA: El boxplot y el histograma muestran una fuerte asimetría positiva y una gran cantidad de valores atípicos (cola derecha pesada). Esto confirma que un modelo que solo predice la media (como un GBR) será insuficiente para gestionar el riesgo de picos de demanda. La idea del modelo híbrido con GPD es correcta.

In [ ]:
# Visualizar 3 series de tiempo de ejemplo
productos_ejemplo = df['prod_id'].unique()[:3]

for pid in productos_ejemplo:
    subset = df[df['prod_id'] == pid]
    plt.plot(subset['fecha'], subset['ventas'], label=f'Producto {pid}')

plt.title('Series de Tiempo (Ejemplo 3 Productos)')
plt.legend()
plt.show()

## 2. Ingeniería de Características (Feature Engineering)
Crearemos características que el modelo pueda usar. Dado que predecimos 2024 (un año completo), no podemos usar lags recientes (como lag_1 o lag_6) porque requerirían predicciones recursivas. Usaremos lags anuales, que son muy robustos para datos con estacionalidad anual.

Crearemos lags anuales (retrasos de 12 y 24 meses) para capturar la estacionalidad sin depender de predicciones recursivas.

In [ ]:
def create_features(data):
    """Crea características de series de tiempo en el dataframe."""
    data = data.sort_values(['prod_id', 'fecha']).copy()
    
    # Features de Calendario
    data['year'] = data['fecha'].dt.year
    data['month'] = data['fecha'].dt.month
    
    # Lags Anuales (robustez para predicción a 12 meses)
    data['lag_12'] = data.groupby('prod_id')['ventas'].shift(12)
    data['lag_13'] = data.groupby('prod_id')['ventas'].shift(13)
    data['lag_24'] = data.groupby('prod_id')['ventas'].shift(24)
    
    # Media móvil sobre el lag para suavizar
    data['rolling_mean_3_lag12'] = data.groupby('prod_id')['lag_12'].transform(lambda x: x.rolling(3).mean())
    
    # Lag del precio (usamos el precio de hace un año como proxy)
    data['precio_lag_12'] = data.groupby('prod_id')['precio_promedio'].shift(12)
    
    return data

# Aplicar la creación de features
df_model = create_features(df)

# Eliminar filas con NaNs generados por los lags
df_model = df_model.dropna()

print("DataFrame con nuevas características:")
print(df_model.shape)
df_model.head()

## 3. Construcción del Modelo (Gradient Boosting)
Definimos el conjunto de entrenamiento y validación. Usaremos todos los datos hasta 2022 para entrenar, y el año 2023 completo para validar.

In [ ]:
# Definir features y target
features = ['month', 'year', 'prod_id', 
            'lag_12', 'lag_13', 'lag_24', 
            'rolling_mean_3_lag12', 'precio_lag_12']
target = 'ventas'

# Dividir Train (<= 2022) y Validation (== 2023)
train = df_model[df_model['year'] < 2023]
val = df_model[df_model['year'] == 2023]

print(f"Filas de entrenamiento: {len(train)}")
print(f"Filas de validación: {len(val)}")

In [ ]:
# Inicializar y entrenar el modelo
print("Entrenando el modelo GBR...")
gbr = GradientBoostingRegressor(
    n_estimators=500,       # Número de árboles
    learning_rate=0.05,     # Tasa de aprendizaje
    max_depth=7,            # Profundidad de los árboles
    random_state=42,
    n_iter_no_change=20,    # Parada temprana (Early Stopping)
    validation_fraction=0.1
)

gbr.fit(train[features], train[target])

print("¡Entrenamiento completado!")

## 4. Guardar el Modelo Entrenado
Guardamos el objeto del modelo GBR en la carpeta models/ para usarlo en el notebook de inferencia (03-model_inference.ipynb).

In [ ]:
# Guardar el modelo entrenado
path_modelo = '../models/gbr_model.joblib'
joblib.dump(gbr, path_modelo)

print(f"Modelo guardado en: {path_modelo}")

## 5. Validación y Comparativa (vs. modelo_actual)
Usamos el set de validación (2023) para medir el rendimiento de nuestro modelo (WAPE y RMSE) y compararlo con el modelo_actual del cliente.

In [ ]:
# 1. Predecir con nuestro nuevo modelo (GBR)
val_preds = gbr.predict(val[features])

# 2. 'val' ya tiene las 'ventas' reales Y el 'modelo_actual'.
#    Solo necesitamos añadirle nuestra nueva predicción.
val_comparativo = val.copy()
val_comparativo['pred_gbr'] = val_preds

# 3. Calcular Métricas
# 'val_comparativo' ahora tiene:
# - 'ventas' (el valor real)
# - 'modelo_actual' (la predicción original, que ya estaba en 'val')
# - 'pred_gbr' (nuestra nueva predicción)

# RMSE (Root Mean Squared Error)
rmse_gbr = np.sqrt(mean_squared_error(val_comparativo['ventas'], val_comparativo['pred_gbr']))
rmse_actual = np.sqrt(mean_squared_error(val_comparativo['ventas'], val_comparativo['modelo_actual']))

# WAPE (Weighted Absolute Percentage Error) - Métrica clave de negocio
wape_gbr = np.sum(np.abs(val_comparativo['ventas'] - val_comparativo['pred_gbr'])) / np.sum(np.abs(val_comparativo['ventas']))
wape_actual = np.sum(np.abs(val_comparativo['ventas'] - val_comparativo['modelo_actual'])) / np.sum(np.abs(val_comparativo['ventas']))

# Resultados
print("--- Comparativa de Modelos (Validación 2023) ---")
print(f"RMSE Modelo Actual: {rmse_actual:.2f}")
print(f"RMSE Nuevo Modelo (GBR): {rmse_gbr:.2f}")
print("\n")
print(f"WAPE Modelo Actual: {wape_actual:.2%}")
print(f"WAPE Nuevo Modelo (GBR): {wape_gbr:.2%}")
print("\n")
mejora_wape = (wape_actual - wape_gbr) / wape_actual
print(f"==> Mejora en WAPE (Precisión de Negocio): {mejora_wape:.2%} ==")

## 6. Guardar Salida de Validación (para Notebook GPD)
Guardamos el val_comparativo (que contiene los errores/residuos) en data/interim/. Este archivo es el input para nuestro notebook 02-model_risk_gpd.ipynb.

In [ ]:
# --- GUARDAR SALIDA PARA MODELO DE RIESGO ---

# El DataFrame 'val_comparativo' contiene todo lo que necesitamos:
# 'ventas' (Reales), 'pred_gbr' (Predicción Core)

# Definir la ruta de salida (siguiendo la estructura Cookiecutter)
path_salida_interim = '../data/interim/model_core_validation.csv'

# Guardar el DataFrame
val_comparativo.to_csv(path_salida_interim, index=False)

print(f"Resultados de validación (con predicciones) guardados en: {path_salida_interim}")
print(val_comparativo.head())

## 5. Generación de Predicciones 2024
Ahora que el modelo está entrenado y validado, creamos el "scaffolding" (andamio) para 2024, aplicamos la ingeniería de características y predecimos.

In [ ]:
# 1. Crear el dataframe futuro para 2024
future_dates = pd.date_range(start='2024-01-01', end='2024-12-01', freq='MS')
prod_ids = df['prod_id'].unique()

df_future_rows = []
for pid in prod_ids:
    for date in future_dates:
        df_future_rows.append({
            'fecha': date,
            'prod_id': pid,
            'ventas': np.nan,          # A predecir
            'precio_promedio': np.nan # No lo usamos directamente
        })

df_future = pd.DataFrame(df_future_rows)

# 2. Concatenar historial + futuro para crear features
df_full = pd.concat([df, df_future], axis=0)

# 3. Aplicar ingeniería de características
df_full_features = create_features(df_full)

# 4. Filtrar solo el 2024 (que ahora tiene features como lag_12)
df_2024 = df_full_features[df_full_features['fecha'].dt.year == 2024].copy()

# 5. Predecir
# (Puede haber nulos en features si un producto es nuevo, por seguridad llenamos con 0)
df_2024[features] = df_2024[features].fillna(0)

df_2024['prediccion_ventas'] = gbr.predict(df_2024[features])

# 6. Guardar el CSV
# En una estructura Cookiecutter, esto iría a 'data/processed/' o 'models/'
path_salida = 'predicciones_demanda_2024.csv' # Ajusta la ruta

output_cols = ['fecha', 'prod_id', 'prediccion_ventas']
df_2024[output_cols].to_csv(path_salida, index=False)

print(f"Predicciones 2024 generadas y guardadas en '{path_salida}'")
df_2024[output_cols].head()

## 6. Próximos Pasos: Hacia el Modelo Híbrido (GBR + GPD)
Este notebook ha construido con éxito un Modelo Core que predice la media de la demanda con mayor precisión (menor WAPE) que el modelo actual.

Sin embargo, como vimos en el EDA, el negocio no está en la media, está en los extremos. Los costos de quiebre de stock (subestimar) y desperdicio (sobreestimar) son asimétricos y ocurren en las colas de la distribución.

El siguiente paso lógico es usar las predicciones de este modelo (pred_gbr) y los valores reales para calcular los residuos (errores).

Residuo = Venta_Real - Predicción_GBR

Sobre la distribución de estos residuos (específicamente, la cola positiva) aplicaremos un modelo de Teoría de Valores Extremos (GPD - Generalized Pareto Distribution). Esto nos permitirá dejar de preguntar "¿Cuánto vamos a vender?" y empezar a preguntar:

"Dado que predecimos vender 1000 unidades (GBR), ¿cuál es el stock de seguridad necesario para tener un 98% de probabilidad de cubrir la demanda real? (GPD)"